# Example usage

To use `my_package_project` in a project:

In [39]:
import my_package_project

print(my_package_project.__version__)

1.3.0


Package import : 

In [40]:
from datetime import date, timedelta, datetime
from my_package_project.data_treatment import *
from my_package_project.graphs  import *
from my_package_project.operations import *

Range of date and universe

By default, the universe is settled as ['SPY', 'TLT', 'GLD'] i.e. S&P 500 Index, 20+ Year Treasury Bond, Gold (commodities) as it is interesting to see the risk contribution of different asset classes but any list of assets available on yfinance can be set as input.

In [41]:
universe = ['SPY', 'TLT', 'GLD']
initial_date = datetime(2005, 1, 1)
final_date = datetime(2020, 12, 31)
data = get_stocks_data(tickers=universe, start_date=initial_date, end_date=final_date).dropna()

#### Data Treatment :

In [42]:
#Methods from Pybacktestchain :
data_module = DataModule(data=data)
info = Information(s = timedelta(data_module.data.size/len(universe)), data_module=data_module)
t = datetime(2020,12,29)
moments = FirstTwoMoments.compute_information(info, t=t)

Compute the Risk Parity portfolio :

In [43]:
riskparityportfolio =  data_treatment.RiskParity.compute_portfolio_riskparity(info, t=t, information_set=moments)
riskparityportfolio

{'GLD': 0.33069177058197635,
 'SPY': 0.1631111191275985,
 'TLT': 0.5061971102904251}

Compute portfolio's volatility : 

In [44]:
vol = data_treatment.portfolio_volatility(riskparityportfolio,information_set=moments)
vol

28.708093200213458

Compute the Risk Parity portfolio with leverage and target volatility : 

In [45]:
data_treatment.RiskParity.compute_portfolio_riskparity_voltarget_leverage(info, t=t, information_set=moments, target_volatility=vol, leverage_factor=1)

INFO:root:Original Volatility: 28.708093200213458


{'GLD': 0.33069177058197635,
 'SPY': 0.1631111191275985,
 'TLT': 0.5061971102904251}

In [46]:
data_treatment.RiskParity.compute_portfolio_riskparity_voltarget_leverage(info, t=t, information_set=moments, target_volatility=15, leverage_factor=2)

INFO:root:Original Volatility: 28.708093200213458


{'GLD': 0.34557339103892576,
 'SPY': 0.17045136156209673,
 'TLT': 0.5289767315023152}

Compute risk contributions :

In [47]:
data_treatment.compute_risk_contributions(riskparityportfolio, moments)

{'GLD': 0.333, 'SPY': 0.333, 'TLT': 0.334}

#### Graphs : 

In [48]:
visualizer = PortfolioVisualizer(portfolio=riskparityportfolio, information_set=moments)

Plot portfolio initial weight :

In [52]:
visualizer.plot_portfolio_weights()

Plot portfolio risk allocation : 

In [53]:
visualizer.plot_risk_allocation_pie()

Methods that enable to plot or compute metrics of a portfolio over time are useful in a backtest such as follows:

#### Running a backtest : 

In [51]:
backtest = Backtest_up(
    initial_date = initial_date,
    final_date = final_date,
    information_class = FirstTwoMoments,
    risk_model=StopLoss,
)
backtest.run_backtest()

INFO:root:Running backtest from 2005-01-01 00:00:00 to 2020-12-31 00:00:00.
INFO:root:Retrieving price data for universe
/Users/marc/Desktop/anaconda3/envs/pybacktestchain/lib/python3.11/site-packages/pandas/core/frame.py:11211: RuntimeWarning:

Degrees of freedom <= 0 for slice

/Users/marc/Desktop/anaconda3/envs/pybacktestchain/lib/python3.11/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning:

divide by zero encountered in divide

/Users/marc/Desktop/anaconda3/envs/pybacktestchain/lib/python3.11/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning:

invalid value encountered in multiply

INFO:root:Stop loss triggered for GLD at 2005-01-04 00:00:00. Selling all shares.
INFO:root:Stop loss triggered for TLT at 2005-01-04 00:00:00. Selling all shares.
INFO:root:-----------------------------------
INFO:root:Rebalancing portfolio at 2005-01-31 00:00:00
INFO:root:-----------------------------------
INFO:root:Rebalancing portfolio at 2005-02-28 00:00:00
INFO:root:Buying

Risk Contributions: {'GLD': 0.333, 'SPY': 0.333, 'TLT': 0.333}


Portfolio Volatility: 29.58%
Annualized Return: 8.18%
Sharpe Ratio: 0.21


INFO:root: Backtest completed. Final portfolio value: 2260623.6384124756


'backtests/GoldCobraPlumber'